In [ ]:
from pyspark.context import SparkContext, SparkConf
from pyspark.sql import *
from pyspark.sql.functions import explode
from pyspark.sql.functions import col, asc
from pyspark.sql.functions import countDistinct
from pyspark.sql.functions import *
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit
from google.cloud.storage.blob import Blob
from google.cloud import storage
import os
import sys
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
import json
import requests
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
import csv

In [ ]:
conf = SparkConf().setAppName('geoloc-cons-project').setMaster('local[4]') 
sc = SparkContext.getOrCreate(conf) 
spark = SparkSession(sc)

print(sc.getConf().getAll())

[('spark.app.id', 'local-1678307730843'), ('spark.sql.catalogImplementation', 'hive'), ('spark.rpc.message.maxSize', '512'), ('spark.rdd.compress', 'True'), ('spark.serializer.objectStreamReset', '100'), ('spark.master', 'local[*]'), ('spark.executor.id', 'driver'), ('spark.submit.deployMode', 'client'), ('spark.driver.host', '70208d7f042b'), ('spark.app.name', 'PySparkShell'), ('spark.ui.showConsoleProgress', 'true'), ('spark.driver.port', '37579')]


In [ ]:
#import registered office details
address_url = 'https://storage.googleapis.com/geolocation_data_project_cm/registered_office_52.csv'
df = pd.read_csv(address_url, delimiter=';', encoding='utf-8')
print(df.head(20))
print(df.info())

    address_seq_no              ADDRESS_L1  \
0               23    ΠΑΝΤΕΛΗ ΚΑΤΕΛΑΡΗ 16    
1               24                ΔΑΜΩΝΟΣ    
2               27                 ΣΙΝΑ 3    
3               30             KENNEDY 12    
4               31    ΔΗΜΟΣΘΕΝΗ ΣΕΒΕΡΗ 17    
5               32              ΠΡΕΣΠΑΣ 2    
6               33             KENNEDY 12    
7               35              ΤΕΜΠΩΝ 6B    
8               36           ΛΥΚΑΒΗΤΟΥ 75    
9               37              ΤΕΜΠΩΝ 6Β    
10              39           ΚΑΛΟΓΡΑΙΩΝ 7    
11              40             ΛΑΡΝΑΚΟΣ 1    
12              44           ΚΑΛΟΓΡΑΙΩΝ 7    
13              47           ΚΑΛΟΓΡΑΙΩΝ 7    
14              48  ΑΡΧ. ΜΑΚΑΡΙΟΥ ΙΙΙ 2-4    
15              49           ΛΥΚΑΒΗΤΟΥ 47    
16              50       ΑΝΝΗΣ ΚΟΜΝΗΝΗΣ 4    
17              51            ΛΑΜΠΟΥΣΑΣ 1    
18              52      ΚΥΡΙΑΚΟΥ ΜΑΤΣΗ 16    
19              53          ΜΕΣΟΛΟΓΓΙΟΥ 6    

                              ADD

In [ ]:
#Address preprocessing to make them fit for analysis
df = df.fillna('')

#Merge useful addresses part 
df['Full_Address'] = df['ADDRESS_L1'] + ' ' + df['ADDRESS_L3']

#Drop unnecessary columns
df = df.drop(['ADDRESS_L1','ADDRESS_L2','ADDRESS_L3'], axis=1)

df.head(20)

,address_seq_no,Full_Address
0,23,ΠΑΝΤΕΛΗ ΚΑΤΕΛΑΡΗ 16 1097 ΛΕΥΚΩΣΙΑ
1,24,ΔΑΜΩΝΟΣ ΜΕΣΑ ΓΕΙΤΟΝΙΑ ΛΕΜΕΣΟΣ
2,27,ΣΙΝΑ 3 1905 ΛΕΥΚΩΣΙΑ
3,30,KENNEDY 12 ΛΕΥΚΩΣΙΑ
4,31,ΔΗΜΟΣΘΕΝΗ ΣΕΒΕΡΗ 17 1080 ΛΕΥΚΩΣΙΑ
5,32,ΠΡΕΣΠΑΣ 2 1082 ΛΕΥΚΩΣΙΑ
6,33,KENNEDY 12 1703 ΛΕΥΚΩΣΙΑ
7,35,ΤΕΜΠΩΝ 6B ΑΓΛΑΝΤΖΙΑ 2114 ΛΕΥΚΩΣΙΑ
8,36,ΛΥΚΑΒΗΤΟΥ 75 ΕΓΚΩΜΗ 2401 ΛΕΥΚΩΣΙΑ
9,37,ΤΕΜΠΩΝ 6Β ΑΓΛΑΝΤΖΙΑ 2114 ΛΕΥΚΩΣΙΑ


In [ ]:
print("Total addresses in dataframe: ",df['address_seq_no'].count())
print("Unique addresses in dataframe: ",df['Full_Address'].nunique())

Total addresses in dataframe:  178916
Unique addresses in dataframe:  124297


In [ ]:
a_1 = df[:5000]
addresses_df1 = spark.createDataFrame(a_1)

In [ ]:
addresses_df1.take(5)

[Row(address_seq_no=23, Full_Address='ΠΑΝΤΕΛΗ ΚΑΤΕΛΑΡΗ 16  1097 ΛΕΥΚΩΣΙΑ '),
 Row(address_seq_no=24, Full_Address='ΔΑΜΩΝΟΣ  ΜΕΣΑ ΓΕΙΤΟΝΙΑ ΛΕΜΕΣΟΣ '),
 Row(address_seq_no=27, Full_Address='ΣΙΝΑ 3  1905 ΛΕΥΚΩΣΙΑ '),
 Row(address_seq_no=30, Full_Address='KENNEDY 12  ΛΕΥΚΩΣΙΑ '),
 Row(address_seq_no=31, Full_Address='ΔΗΜΟΣΘΕΝΗ ΣΕΒΕΡΗ 17  1080 ΛΕΥΚΩΣΙΑ ')]

In [ ]:
# Authenticate and set the API Key
apikey = 'AIzaxxxxxxxxxxxxxxxxx'

with open('output_nan.csv', 'a', encoding='utf-8') as g:
    writer = csv.writer(g)
    writer.writerow(['seq_no', 'coordinates', 'lat', 'lng'])
    
with open('output_coords.csv', 'a', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['seq_no', 'coordinates', 'lat', 'lng'])

def get_coordinates(seq_no, address):

    url = f'https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={apikey}' 
    response = requests.get(url).json() 
    if response['status'] == 'ZERO_RESULTS':
        lat = 'NaN'
        lng = 'NaN'
        with open('output_nan.csv', 'a', encoding='utf-8') as g:
            writer = csv.writer(g)
            writer.writerow([seq_no, (lat, lng), lat, lng])
        return lat, lng
    else:
        lat = response['results'][0]['geometry']['location']['lat']
        lng = response['results'][0]['geometry']['location']['lng']
        with open('output_coords.csv', 'a', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow([seq_no, (lat, lng), lat, lng])
        return lat, lng

# Get the first 1000 records
offset = 0
batch_size = 500
max_records = 5000

while (offset<=5000):
    btch_df = addresses_df1.filter((col("address_seq_no") >= offset) & (col("address_seq_no") < (offset+batch_size)))
    rdd = btch_df.rdd.map(lambda x: get_coordinates(x['address_seq_no'], x['Full_Address']))
    work_df = spark.createDataFrame(rdd, schema=['address_seq_no', 'lat_lng'])
    work_df.show()

    if btch_df.count() < batch_size:
        break
    else:
        offset += batch_size


23/03/07 20:19:14 WARN org.apache.spark.scheduler.TaskSetManager: Stage 13 contains a task of very large size (149 KB). The maximum recommended task size is 100 KB.


+--------------+----------+
|address_seq_no|   lat_lng|
+--------------+----------+
|    35.1626365|33.3568121|
|    35.1681853|33.3654403|
|     35.022874|33.3947933|
|    35.1729524|33.3808787|
|    35.1721664|33.3628335|
|    34.6862007|33.0529321|
|    34.6862007|33.0529321|
|    35.1656389|33.3593045|
|     34.685046|33.0305082|
|    34.8646102|33.4894387|
|    35.1336349|33.3484948|
|     35.118736|33.3830023|
|    35.1071472|33.3843009|
|    34.9187642|33.6364169|
|    34.9204205|33.6180488|
|    35.1722806|33.3764188|
|    34.7636776| 32.428295|
|    35.1468344|33.3734787|
|     34.688008| 33.052927|
|    34.6786322|33.0413055|
+--------------+----------+
only showing top 20 rows



In [ ]:
df['address_seq_no'][5000]

12101

In [ ]:
# Get the next 4000 records
offset = 1000
batch_size = 500
max_records = 5000
max_ids = df['address_seq_no'][5000]

while (offset<=max_ids):
    df_name = 'new_df_' + str(offset)
    rdd_name = 'rdd_' + str(offset)
    work_name = 'work_' + str(offset)
    globals()[df_name] = addresses_df1.filter((col("address_seq_no") >= offset) & (col("address_seq_no") < (offset+batch_size)))
    globals()[rdd_name] = globals()[df_name].rdd.map(lambda x: get_coordinates(x['address_seq_no'], x['Full_Address']))
    globals()[work_name] = spark.createDataFrame(globals()[rdd_name], schema=['address_seq_no', 'lat_lng'])
    globals()[work_name].show()

    if offset > max_ids:
        break
    else:
        offset = offset + batch_size


23/03/07 21:00:35 WARN org.apache.spark.scheduler.TaskSetManager: Stage 55 contains a task of very large size (149 KB). The maximum recommended task size is 100 KB.
23/03/07 21:00:35 WARN org.apache.spark.scheduler.TaskSetManager: Stage 57 contains a task of very large size (149 KB). The maximum recommended task size is 100 KB.
23/03/07 21:00:57 WARN org.apache.spark.scheduler.TaskSetManager: Stage 59 contains a task of very large size (149 KB). The maximum recommended task size is 100 KB.


+--------------+----------+
|address_seq_no|   lat_lng|
+--------------+----------+
|    34.7079571|33.0091404|
|    34.6832414|33.0451419|
|    34.9899859|32.9036838|
|    35.1536584|33.3927867|
|    35.1227622|33.3197245|
|    34.7587825|32.9929063|
|    34.7258619|33.1404129|
|    34.6946276|33.0393926|
|    34.6836382|33.0519284|
|    35.1602214|33.3604734|
|          null|      null|
|    35.1721359|33.3475341|
|    35.1566889|33.3299703|
|    34.7008253|33.0759934|
|    35.0151025|33.3944356|
|    34.7774173|32.4410944|
|    35.1510316|33.3598978|
|    35.1684673|33.3339106|
|    34.7751463|32.4197039|
|    34.7587825|32.9929063|
+--------------+----------+
only showing top 20 rows



23/03/07 21:00:57 WARN org.apache.spark.scheduler.TaskSetManager: Stage 61 contains a task of very large size (149 KB). The maximum recommended task size is 100 KB.
23/03/07 21:01:20 WARN org.apache.spark.scheduler.TaskSetManager: Stage 63 contains a task of very large size (149 KB). The maximum recommended task size is 100 KB.


+-----------------+----------+
|   address_seq_no|   lat_lng|
+-----------------+----------+
|34.67921490000001|33.0416677|
|       35.1028182|33.0941709|
|       34.6964607|33.0693123|
|       34.6982601|33.0455093|
|       35.1436629|33.3437909|
|       34.6786322|33.0413055|
|       34.7839033|32.4219428|
|       34.7948464| 32.437597|
|       35.1654533|33.3612831|
|       35.1730282|33.3809027|
|       35.1589658|33.3197357|
|35.16445849999999|33.3319528|
|       35.1498196|33.3685283|
|       35.1699512| 33.356571|
|       34.6831514|32.9903099|
|       35.1724934|33.3508949|
|       35.1699512| 33.356571|
|       34.7116805|33.0228317|
|       34.6652051|33.0088478|
|        35.161849|33.3567025|
+-----------------+----------+
only showing top 20 rows



23/03/07 21:01:21 WARN org.apache.spark.scheduler.TaskSetManager: Stage 65 contains a task of very large size (149 KB). The maximum recommended task size is 100 KB.
23/03/07 21:01:42 WARN org.apache.spark.scheduler.TaskSetManager: Stage 67 contains a task of very large size (149 KB). The maximum recommended task size is 100 KB.


...

+--------------+----------+
|address_seq_no|   lat_lng|
+--------------+----------+
|    34.7587825|32.9929063|
|     34.689381|33.0087394|
|    34.6982601|33.0455093|
|    34.9922985| 33.682669|
|    35.1574329|33.3437194|
|    34.7530522|33.0983787|
|    35.1546896|33.3996153|
|     34.788292|32.4233068|
|    35.1396127|33.3294161|
|    35.0445032|33.1551245|
|    34.6786322|33.0413055|
|    34.6942622|33.0408942|
|    35.1108902|33.2997139|
|    35.1610307| 33.314904|
|    35.1662005| 33.327161|
|    34.8464781|33.5727305|
|    34.7747873|32.4285134|
|    35.1104088|33.3836088|
|    34.6889729|33.0337738|
|    34.6842799|33.0518155|
+--------------+----------+
only showing top 20 rows



23/03/07 21:04:31 WARN org.apache.spark.scheduler.TaskSetManager: Stage 105 contains a task of very large size (149 KB). The maximum recommended task size is 100 KB.
23/03/07 21:04:49 WARN org.apache.spark.scheduler.TaskSetManager: Stage 107 contains a task of very large size (149 KB). The maximum recommended task size is 100 KB.


+--------------+-----------------+
|address_seq_no|          lat_lng|
+--------------+-----------------+
|    35.1581762|       33.3450638|
|    34.6971934|        33.054231|
|    35.1752816|        33.342647|
|    35.1676253|       33.3361078|
|    35.1855659|33.38227639999999|
|    35.1626365|       33.3568121|
|    35.1689332|       33.3603387|
|    35.0369323|       33.9835815|
|    34.6806316|       33.0471355|
|    34.7587825|       32.9929063|
|    34.7390692|       33.2012827|
|    34.9141903|       33.6109788|
|    34.9208153|       33.6313352|
|    34.9393189|       33.6161497|
|    35.1698311|       33.3570624|
|    35.1700454|       33.3606766|
|    35.1536554|       33.3587054|
|    35.1623205|       33.3259115|
|    34.7002343|       33.0974615|
|    35.0339441|       32.4253751|
+--------------+-----------------+
only showing top 20 rows



23/03/07 21:04:50 WARN org.apache.spark.scheduler.TaskSetManager: Stage 109 contains a task of very large size (149 KB). The maximum recommended task size is 100 KB.
[Stage 109:======================================>                  (2 + 1) / 3]

+-----------------+----------+
|   address_seq_no|   lat_lng|
+-----------------+----------+
|       34.9036406|33.6362153|
|       35.1524256|33.3452233|
|       35.1524256|33.3452233|
|       34.9149968|33.6227325|
|       34.9512484|33.5921052|
|       35.1507057|33.3595431|
|35.10866439999999| 33.422395|
|       34.7258619|33.1404129|
|       34.6718096|33.0124115|
|       35.1014864|33.4072244|
|       35.0999766|33.3835781|
|       35.1516232|33.3601814|
|       34.7686716| 32.436811|
|       34.6786322|33.0413055|
|       35.1608613|33.3533984|
|       34.7017905|33.0410206|
|       34.9247448|33.6261297|
|       34.6979894|33.0116066|
|        34.673301|33.0102573|
|       34.6879576|33.0365459|
+-----------------+----------+
only showing top 20 rows



In [ ]:
a_2 = df[5001:10000]
addresses_df2 = spark.createDataFrame(a_2)
addresses_df2.show()
min_id = df['address_seq_no'][5001]
max_id = df['address_seq_no'][10000]

+--------------+--------------------+
|address_seq_no|        Full_Address|
+--------------+--------------------+
|         12102|ΗΒΗΣ 3  ΑΓΙΟΣ  ΣΠ...|
|         12104|ΒΥΖΑΝΤΙΟΥ 59  ΛΕΥ...|
|         12105|ΚΑΡΑΙΣΚΑΚΗ 38Ζ  Λ...|
|         12107|ΦΛΩΡΙΝΗΣ 11  1065...|
|         12110|ΑΡΜΕΝΙΑΣ 75Α  ΛΕΥ...|
|         12112|ΑΡΧ. ΜΑΚΑΡΙΟΥ ΙΙΙ...|
|         12113|ΑΝΕΞΑΡΤΗΣΙΑΣ 32Α ...|
|         12115|ΑΡΧ. ΜΑΚΑΡΙΟΥ ΙΙΙ...|
|         12116|ΑΡΤΕΜΙΟΥ ΦΡΑΓΚΙΣΚ...|
|         12117|  ΝΑΞΟΥ 4  ΛΕΥΚΩΣΙΑ |
|         12119|JOHN KENNEDY 3  1...|
|         12120|ΚΙΜΩΝΟΣ 21  ΛΑΡΝΑΚΑ |
|         12124|ΑΔΑΜΑΝΤΟΣ 18Α  ΔΕ...|
|         12125|ΞΕΝΟΦΩΝΤΟΣ 7  ΛΑΚ...|
|         12128|ΚΩΝΣΤΑΝΤΙΝΟΥ ΠΑΛΑ...|
|         12130|ΓΡΗΓΟΡΙΟΥ ΛΑΜΠΡΑΚ...|
|         12132|ΓΡΕΒΕΝΩΝ 10  Κ. Π...|
|         12133| ΑΓΛΑΝΤΖΙΑ ΛΕΥΚΩΣΙΑ |
|         12136|ΑΧΙΛΛΕΩΣ 31  ΣΤΡΟ...|
|         12137|ΠΑΝΤΑΙΝΟΥ 6Α  ΣΤΡ...|
+--------------+--------------------+
only showing top 20 rows



In [ ]:
offset2 = 12000
batch_size2 = 500

while (offset2<=max_id):
    df_name2 = 'new_df_' + str(offset2)
    rdd_name2 = 'rdd_' + str(offset2)
    work_name2 = 'work_' + str(offset2)
    globals()[df_name2] = addresses_df2.filter((col("address_seq_no") >= offset2) & (col("address_seq_no") < (offset2+batch_size2)))
    globals()[rdd_name2] = globals()[df_name2].rdd.map(lambda x: get_coordinates(x['address_seq_no'], x['Full_Address']))
    globals()[work_name2] = spark.createDataFrame(globals()[rdd_name2], schema=['address_seq_no', 'lat_lng'])
    globals()[work_name2].show()

    if offset2 > max_id:
        break
    else:
        offset2 = offset2 + batch_size2


+-----------------+-----------------+
|   address_seq_no|          lat_lng|
+-----------------+-----------------+
|       34.6708671|       33.0115108|
|       35.1733629|33.34701520000001|
|       34.6823541|        33.047927|
|       35.1649108|        33.358494|
|35.15426679999999|       33.3642849|
|         35.08804|       33.2784486|
|        34.676214|       33.0456169|
|       34.7587825|       32.9929063|
|       35.1436629|       33.3437909|
|       35.1630664|       33.3665425|
|       35.1723563|        33.374509|
|       34.9208153|       33.6313352|
|        35.062854|        33.960628|
|       35.1185656|       33.3145382|
|       35.1704816|       33.3679515|
|       34.8761451|       33.5857563|
|       34.6809413|       32.9945236|
|       35.1498034|        33.394086|
|       35.1436629|       33.3437909|
|       35.1256701|33.32927859999999|
+-----------------+-----------------+
only showing top 20 rows



[Stage 113:>                                                        (0 + 1) / 1]

+--------------+-----------------+
|address_seq_no|          lat_lng|
+--------------+-----------------+
|    34.7587825|       32.9929063|
|    34.6851915|       33.0560735|
|    34.9521888|       33.5908529|
|    35.1784529|       33.3291115|
|    35.1748103|       33.3477679|
|    34.7587825|       32.9929063|
|    34.6893833|       33.0457155|
|    34.7013709|       33.0941263|
|    34.9288025|       33.6183202|
|    34.7851476|       32.4420334|
|     35.150733|       33.3979417|
|    35.1855659|33.38227639999999|
|    34.9761878|       33.8487527|
|    35.1855659|33.38227639999999|
|    35.1646109|       33.3270002|
|    34.7600248|       32.4533949|
|    35.1059975|       33.3947237|
|    34.7936584|       32.4342298|
|          null|             null|
|    34.6856261|       33.0362065|
+--------------+-----------------+
only showing top 20 rows



+--------------+-----------------+
|address_seq_no|          lat_lng|
+--------------+-----------------+
|    35.1650671|       33.3585292|
|          null|             null|
|    35.1562975|       33.3360956|
|    35.1714101|       33.3732796|
|    34.6824496|       33.0440797|
|    35.1235943|       33.2951713|
|    34.6732829|       33.0230275|
|    34.6648569|        33.014746|
|    35.1560654|       33.3287686|
|    35.1443205|       33.3576107|
|    35.1855659|33.38227639999999|
|    35.1635857|33.35871240000001|
|          null|             null|
|    34.6841082|       33.0397899|
|    35.1705679|       33.3824254|
|    34.6898114|       33.0511322|
|    34.6786322|       33.0413055|
|    35.1698751|       33.3566759|
|    35.1611826|       33.3617769|
|    35.1536324|       33.3433308|
+--------------+-----------------+
only showing top 20 rows



+-----------------+----------+
|   address_seq_no|   lat_lng|
+-----------------+----------+
|       35.1648906|33.3586214|
|       34.7587825|32.9929063|
|       35.1672794|33.3682083|
|       35.0284699|33.9513191|
|        35.162052|33.3672066|
|       35.1577869|33.3540286|
|       34.6810429|33.0303898|
|       35.1602214|33.3604734|
|       35.0195632|33.4192946|
|        34.935371|33.5811995|
|       34.9253248|33.6316475|
|        34.676624|33.0466649|
|       35.1642899|33.3647896|
|       34.7074129|33.0500561|
|       34.8038652|33.4872724|
|       34.7006666|33.0108949|
|       34.9182222|33.6200625|
|       34.7901954|32.4343953|
|       34.6817393|33.0437495|
|35.16702009999999|33.3578493|
+-----------------+----------+
only showing top 20 rows



...

23/03/07 21:28:47 WARN org.apache.spark.scheduler.TaskSetManager: Stage 205 contains a task of very large size (153 KB). The maximum recommended task size is 100 KB.


+--------------+-----------------+
|address_seq_no|          lat_lng|
+--------------+-----------------+
|    34.7743487|       32.4333417|
|    35.1773118|       33.3889478|
|    34.6800562|       33.0205925|
|    34.6871026|       33.0605382|
|    35.1120585|        33.290836|
|    35.1682018|        33.364938|
|    35.1645988|       33.3961141|
|    34.9152094|       33.6047536|
|    34.7587825|       32.9929063|
|     35.128775|       33.3672008|
|    34.6718013|       33.0115043|
|    35.1694615|       33.3567187|
|    35.1747109|       33.3232998|
|    35.1398152|33.33785050000001|
|     35.173278|       33.3593142|
|    35.1337247|       33.3050015|
|    35.1560862|       33.3576668|
|     35.143945|       33.3136545|
|    35.1151718|       33.4195948|
|    34.6786322|       33.0413055|
+--------------+-----------------+
only showing top 20 rows



23/03/07 21:29:06 WARN org.apache.spark.scheduler.TaskSetManager: Stage 207 contains a task of very large size (153 KB). The maximum recommended task size is 100 KB.
23/03/07 21:29:06 WARN org.apache.spark.scheduler.TaskSetManager: Stage 209 contains a task of very large size (153 KB). The maximum recommended task size is 100 KB.
23/03/07 21:29:10 WARN org.apache.spark.scheduler.TaskSetManager: Stage 211 contains a task of very large size (153 KB). The maximum recommended task size is 100 KB.


+-----------------+-----------------+
|   address_seq_no|          lat_lng|
+-----------------+-----------------+
|35.03385129999999|       33.9779562|
|       34.8833497|       32.3874598|
|       34.6786322|       33.0413055|
|       34.9193011|       33.6315054|
|       35.1150785|       33.3115207|
|       34.9887366|       34.0002782|
|       34.9908716|        34.001844|
|       35.1672846|       33.3592077|
|       35.1571886|       33.3891184|
|       34.6741398|       33.0400706|
|       34.6762497|       33.0162334|
|       34.7042417|       33.0867474|
|       34.6700252|       33.0121266|
|       34.9320591|       33.5970006|
|       35.1855659|33.38227639999999|
|       34.9532469|       33.6003461|
|       35.1599229|       33.3589199|
|       35.1434626|       33.3682521|
|       34.8959786|         32.96608|
|       35.1400602|       33.3769555|
+-----------------+-----------------+
only showing top 20 rows



ValueError: RDD is empty

In [ ]:
a_3 = df[10001:15000]
addresses_df3 = spark.createDataFrame(a_3)
addresses_df3.show()
min_id3 = df['address_seq_no'][10001]
max_id3 = df['address_seq_no'][15000]
print(min_id3)
print(max_id3)

+--------------+--------------------+
|address_seq_no|        Full_Address|
+--------------+--------------------+
|         30234|ΘΕΜ. ΔΕΡΒΗ & ΦΛΩΡ...|
|         30235|ΣΠΑΡΤΗΣ 3Α  ΛΑΡΝΑΚΑ |
|         30239| ΣΚΟΥΛΛΙ 8728 ΠΑΦΟΣ |
|         30243|ΓΕΩΡΓΙΟΥ ΣΟΥΡΡΗ 4...|
|         30244|ΣΤΡΟΒΟΛΟΥ 80  ΛΑΚ...|
|         30246|ΘΕΣΣΑΛΙΑΣ 4  ΛΑΚΑ...|
|         30247|ΚΛΗΜΕΝΤΟΣ 8Α  106...|
|         30248|ΓΕΩΡΓΙΟΥ ΝΕΟΦΥΤΟΥ...|
|         30249|ΓΑΛΙΛΑΙΟΥ 2  ΑΓΙΟ...|
|         30250|ΘΕΜΙΣΤΟΚΛΗ ΔΕΡΒΗ ...|
|         30251|ΜΑΚΑΡΙΟΥ Γ΄  ΄ΥΨΩ...|
|         30252|ΑΝΕΞΑΡΤΗΣΙΑΣ 165 ...|
|         30253|1ΗΣ ΑΠΡΙΛΙΟΥ 55  ...|
|         30255|ΑΝΕΞΑΡΤΗΣΙΑΣ 11  ...|
|         30261|ΠΛΟΥΤΩΝΟΣ 7  ΄ΑΓΙ...|
|         30264|ΕΥΑΓΟΡΑ ΠΑΛΛΗΚΑΡΙ...|
|         30265|ΚΛΕΩΝΟΣ 7  3085 Λ...|
|         30266|ΜΙΧΑΛΑΚΗ ΚΥΠΡΙΑΝΟ...|
|         30267|ΜΑΚΑΡΙΟΥ 22  Μ. Γ...|
|         30268|ΘΕΚΛΑΣ ΛΥΣΣΙΩΤΟΥ ...|
+--------------+--------------------+
only showing top 20 rows

30234
45408


In [ ]:
offset3 = 30000
batch_size3 = 500

while (offset3<=max_id3):
    df_name3 = 'new_df_' + str(offset3)
    rdd_name3 = 'rdd_' + str(offset3)
    work_name3 = 'work_' + str(offset3)
    globals()[df_name3] = addresses_df3.filter((col("address_seq_no") >= offset3) & (col("address_seq_no") < (offset3+batch_size3)))
    globals()[rdd_name3] = globals()[df_name3].rdd.map(lambda x: get_coordinates(x['address_seq_no'], x['Full_Address']))
    globals()[work_name3] = spark.createDataFrame(globals()[rdd_name3], schema=['address_seq_no', 'lat_lng'])
    globals()[work_name3].show()

    if offset3 > max_id3:
        break
    else:
        offset3 = offset3 + batch_size3

+--------------+-----------------+
|address_seq_no|          lat_lng|
+--------------+-----------------+
|           NaN|              NaN|
|    34.9105843|       33.6294324|
|    34.9805895|       32.4511485|
|    34.7812066|       32.4321558|
|    35.1456858|       33.3447454|
|    35.1185656|       33.3145382|
|    35.1677416|       33.3695557|
|    34.6961524|       33.0624667|
|    34.6724389|       33.0204641|
|    35.1635857|33.35871240000001|
|    34.7074492|       33.0492863|
|    34.6797016|        33.043696|
|    35.0415791|        33.978529|
|    34.7740259|       32.4402864|
|    34.6912201|       33.0552544|
|    34.7784847|       32.4243114|
|    34.6885282|       33.0337719|
|    34.8721676|       32.3780072|
|    34.6786322|       33.0413055|
|    34.6857657|       33.0475219|
+--------------+-----------------+
only showing top 20 rows



+-----------------+----------+
|   address_seq_no|   lat_lng|
+-----------------+----------+
|        35.167755|33.3640957|
|       35.0195632|33.4192946|
|       35.1436629|33.3437909|
|       35.0375882|33.9814728|
|34.98983279999999|33.9948787|
|       35.1180451|33.3776563|
|       34.6786322|33.0413055|
|       34.7888153|32.9791008|
|       34.6564363|33.0029105|
|       34.6804758|32.9383663|
|       35.1727869|33.3290916|
|        34.990751|34.0009402|
|       35.0409093|33.9267967|
|       34.9366546|33.6052384|
|       35.0375882|33.9814728|
|       35.1427805|33.3740234|
|       35.1721438|33.3746948|
|       35.1527991|33.1999632|
|       35.1538337|33.3340108|
|       34.9161665|33.6005853|
+-----------------+----------+
only showing top 20 rows



+-----------------+-----------------+
|   address_seq_no|          lat_lng|
+-----------------+-----------------+
|       34.9761878|       33.8487527|
|       34.6786322|       33.0413055|
|       35.1855659|33.38227639999999|
|       34.6946383|       33.0330445|
|       35.1604146|       33.3531336|
|       34.9165748|       33.6259971|
|        34.666399|       33.0045882|
|       34.7016169|       32.9956475|
|34.77539489999999|       32.4217786|
|       34.9147583|        33.637831|
|         34.68802|       33.0365322|
|             null|             null|
|       35.1108799|       33.2971592|
|       35.1784783|       33.3789925|
|        34.680048|       33.0175155|
|       34.7203043|       33.0198024|
|       35.1610337|       33.3605995|
|       35.0726358|       33.3948138|
|       34.6875677|       33.0550146|
|       34.7839043|       32.4219464|
+-----------------+-----------------+
only showing top 20 rows



+--------------+-----------------+
|address_seq_no|          lat_lng|
+--------------+-----------------+
|    35.1618672|       33.3649711|
|    35.1149116|        33.919245|
|    35.1534883|       33.3746208|
|    34.6835682|       33.0087394|
|    34.6943926|       33.0336115|
|    35.1855659|33.38227639999999|
|     34.689381|       33.0087394|
|    34.7600248|       32.4533949|
|    35.1855659|33.38227639999999|
|    34.7750732|       32.4384647|
|    34.6761854|        33.005703|
|    35.0096685|       33.8929049|
|    34.9156015|       33.6100146|
|    34.7654893|       32.4187639|
|    35.1681853|       33.3654403|
|    34.6903303|         33.04463|
|    35.1571097|       33.3923408|
|    34.9155459|       33.6350037|
|     35.163212|       33.4001544|
|    35.1128873|       33.2928089|
+--------------+-----------------+
only showing top 20 rows



+--------------+----------+
|address_seq_no|   lat_lng|
+--------------+----------+
|    35.1727869|33.3290916|
|    35.1195615|33.3265392|
|    35.0522614|33.9857834|
|    35.1637522|33.3223534|
|    34.7936584|32.4342298|
|    34.9183804|33.6330289|
|    34.6857678|33.0223791|
|    34.7809147|32.4317948|
|     34.729241|32.7788665|
|    34.9887366|34.0002782|
|    34.9170601|33.6228664|
|    34.9182222|33.6200625|
|    34.9222965|33.6080763|
|    34.9149055|33.6231018|
|    35.0162776| 33.392085|
|    35.1729524|33.3808787|
|    34.6860519|33.0631665|
|    34.6786322|33.0413055|
|    35.1436629|33.3437909|
|    34.6704716|32.9021979|
+--------------+----------+
only showing top 20 rows



...

23/03/07 21:45:09 WARN org.apache.spark.scheduler.TaskSetManager: Stage 251 contains a task of very large size (148 KB). The maximum recommended task size is 100 KB.
23/03/07 21:45:26 WARN org.apache.spark.scheduler.TaskSetManager: Stage 253 contains a task of very large size (148 KB). The maximum recommended task size is 100 KB.


+--------------+----------+
|address_seq_no|   lat_lng|
+--------------+----------+
|    35.1461413|33.3486477|
|    35.1666698|33.3695295|
|    35.1526733|33.3873619|
|    35.1433264|33.3787198|
|    34.9506826|33.4287225|
|    34.6789184|32.9945292|
|    34.6697128|32.7006261|
|    34.9341099|33.6268437|
|    35.1446934|33.3446408|
|    34.9398115| 33.613894|
|    35.1470784|33.3570478|
|    34.9117588|33.6349571|
|    34.9521888|33.5908529|
|    35.1660942|33.3512452|
|    35.1306701|33.3174026|
|    35.0452077|33.9782591|
|    35.1682426|33.3620429|
|    35.1570345|33.3101002|
|    35.0153362|33.7407024|
|    35.1105161|33.2878551|
+--------------+----------+
only showing top 20 rows



23/03/07 21:45:27 WARN org.apache.spark.scheduler.TaskSetManager: Stage 255 contains a task of very large size (148 KB). The maximum recommended task size is 100 KB.
23/03/07 21:45:39 WARN org.apache.spark.scheduler.TaskSetManager: Stage 257 contains a task of very large size (148 KB). The maximum recommended task size is 100 KB.


+-----------------+----------+
|   address_seq_no|   lat_lng|
+-----------------+----------+
|       34.6871158|33.0543869|
|       34.6967913| 33.060166|
|             null|      null|
|       35.0057783|33.1622181|
|       35.0377476|33.9224763|
|       34.6753811| 33.044867|
|       34.7071604| 33.013908|
|       35.0991923|33.3770103|
|       35.0455495|33.9840935|
|       34.6948888|33.0431682|
|       35.1612433|33.3538391|
|       34.6786322|33.0413055|
|35.16783059999999| 33.351667|
|       35.1655348|33.3640062|
|       35.1697054|33.3574257|
|       34.6885427|33.0439557|
|        35.151055|33.3595158|
|       35.1731822|33.3794196|
|       35.1421702|33.3511225|
|       35.1742143|33.3794717|
+-----------------+----------+
only showing top 20 rows



ValueError: RDD is empty

In [ ]:
a_4 = df[15001:30000]
addresses_df4 = spark.createDataFrame(a_4)
addresses_df4.show()
min_id4 = df['address_seq_no'][15001]
max_id4 = df['address_seq_no'][30000]
print(min_id4)
print(max_id4)


+--------------+--------------------+
|address_seq_no|        Full_Address|
+--------------+--------------------+
|         45409|ΝΙΚΟΥ ΠΑΤΤΙΧΗ 37 ...|
|         45410|ΛΕΩΝΙΔΟΥ 6Α  ΑΚΡΟ...|
|         45412|ΨΗΛΟΡΕΙΤΗ 28Α  60...|
|         45415|ΑΓΙΑΣ ΕΛΕΝΗΣ 36  ...|
|         45416|ΟΜΗΡΟΥ 4  ΔΑΛΙ ΛΕ...|
|         45417| ΚΑΚΟΠΕΤΡΙΑ 2800 ...|
|         45418|ΑΡΧ. ΜΑΚΑΡΙΟΥ ΙΙΙ...|
|         45419|ΝΗΣΙ 6  ΑΓΙΑ ΝΑΠΑ...|
|         45420|ΠΑΤΡΩΝ ΓΕΡΜΑΝΟΥ 1...|
|         45422| ΑΘΗΝΩΝ 30  ΛΑΡΝΑΚΑ |
|         45424|ΕΛΕΟΥΣΗΣ 16  ΣΥΝΟ...|
|         45426|ΜΕΤΟΧΙΟΥ 11  ΟΡΜΗ...|
|         45429|ΠΥΘΑΓΟΡΑ 12  ΑΓΙΑ...|
|         45430|ΑΡΣΙΝΟΗΣ 55  ΑΚΡΟ...|
|         45433|ΕΛΕΥΘΕΡΙΟΥ ΒΕΝΙΖΕ...|
|         45437|ΑΓΙΟΥ ΠΑΥΛΟΥ 45  ...|
|         45439|ΧΡΥΣΑΝΘΟΥ ΜΥΛΩΝΑ ...|
|         45440|  ΛΗΔΑΣ 2Α  ΛΕΜΕΣΟΣ |
|         45441|ΧΡΥΣΑΝΘΟΥ ΜΥΛΩΝΑ ...|
|         45442|ΡΗΓΑ ΦΕΡΑΙΟΥ 2  Λ...|
+--------------+--------------------+
only showing top 20 rows

45409
52596


23/03/07 21:56:26 WARN org.apache.spark.scheduler.TaskSetManager: Stage 259 contains a task of very large size (233 KB). The maximum recommended task size is 100 KB.


In [ ]:
offset4 = 45400
batch_size4 = 500

while (offset4<=max_id4):
    df_name4 = 'new_df_' + str(offset4)
    rdd_name4 = 'rdd_' + str(offset4)
    work_name4 = 'work_' + str(offset4)
    globals()[df_name4] = addresses_df4.filter((col("address_seq_no") >= offset4) & (col("address_seq_no") < (offset4+batch_size4)))
    globals()[rdd_name4] = globals()[df_name4].rdd.map(lambda x: get_coordinates(x['address_seq_no'], x['Full_Address']))
    globals()[work_name4] = spark.createDataFrame(globals()[rdd_name4], schema=['address_seq_no', 'lat_lng'])
    globals()[work_name4].show()

    if offset4 > max_id4:
        break
    else:
        offset4 = offset4 + batch_size4

23/03/07 21:58:46 WARN org.apache.spark.scheduler.TaskSetManager: Stage 260 contains a task of very large size (233 KB). The maximum recommended task size is 100 KB.
23/03/07 21:58:46 WARN org.apache.spark.scheduler.TaskSetManager: Stage 261 contains a task of very large size (233 KB). The maximum recommended task size is 100 KB.
23/03/07 21:59:21 WARN org.apache.spark.scheduler.TaskSetManager: Stage 262 contains a task of very large size (233 KB). The maximum recommended task size is 100 KB.


+--------------+-----------------+
|address_seq_no|          lat_lng|
+--------------+-----------------+
|    34.6841814|       33.0291831|
|    35.1692187|       33.3590218|
|    34.9359227|       33.6368164|
|      35.16643|       33.3621863|
|    35.0195632|       33.4192946|
|    34.9899859|       32.9036838|
|    34.6982601|       33.0455093|
|     34.988564|       33.9854609|
|    34.6885818|       33.0463068|
|    34.9135615|       33.6377305|
|    34.6980679|       33.0831628|
|    34.9923198|       33.7791944|
|    34.6851842|         33.04416|
|    35.1554011|       33.3581484|
|    35.0617995|       33.5399098|
|    35.1855659|33.38227639999999|
|    34.6862007|       33.0529321|
|    34.6706468|       33.0107403|
|    34.6862007|       33.0529321|
|    34.6871048|        33.053674|
+--------------+-----------------+
only showing top 20 rows



23/03/07 21:59:21 WARN org.apache.spark.scheduler.TaskSetManager: Stage 263 contains a task of very large size (233 KB). The maximum recommended task size is 100 KB.
23/03/07 21:59:53 WARN org.apache.spark.scheduler.TaskSetManager: Stage 264 contains a task of very large size (233 KB). The maximum recommended task size is 100 KB.


+-----------------+----------+
|   address_seq_no|   lat_lng|
+-----------------+----------+
|35.18710410000001|33.3784958|
|       35.1268061|33.1921206|
|       35.1731822|33.3794196|
|        34.916389|33.6366674|
|       35.1568169|33.3445964|
|       34.6870513|33.0441326|
|       35.1712318|33.3605223|
|       34.6871048| 33.053674|
|       35.0201491|33.4229704|
|       34.6786322|33.0413055|
|             null|      null|
|       34.6682844|33.0091425|
|       35.1640951|33.3490875|
|       35.1369868|33.3512888|
|       34.9217597|33.6278297|
|       34.6982601|33.0455093|
|       34.6981303| 33.093928|
|       35.1698273|33.3392768|
|34.77539489999999|32.4217786|
|        35.165442|33.3650498|
+-----------------+----------+
only showing top 20 rows



23/03/07 21:59:53 WARN org.apache.spark.scheduler.TaskSetManager: Stage 265 contains a task of very large size (233 KB). The maximum recommended task size is 100 KB.
23/03/07 22:00:30 WARN org.apache.spark.scheduler.TaskSetManager: Stage 266 contains a task of very large size (233 KB). The maximum recommended task size is 100 KB.


...

+--------------+----------+
|address_seq_no|   lat_lng|
+--------------+----------+
|    35.1752596|33.3866209|
|    34.6752126|33.0434174|
|    34.9182222|33.6200625|
|    34.7109373|33.0597452|
|    34.7587825|32.9929063|
|    34.7220992|33.0839754|
|    34.7754278|32.4274806|
|    34.6786322|33.0413055|
|    35.0441187|33.9982224|
|    34.6786322|33.0413055|
|    35.1727869|33.3290916|
|    34.6914176|33.0354757|
|    34.7109373|33.0597452|
|    34.6818797| 33.051031|
|    35.1755831| 33.330922|
|    34.8548144|33.5468888|
|    35.1480203|33.3633068|
|    34.8256084|33.3893174|
|    34.6786322|33.0413055|
|    34.7817976|32.4382504|
+--------------+----------+
only showing top 20 rows



23/03/07 22:05:49 WARN org.apache.spark.scheduler.TaskSetManager: Stage 296 contains a task of very large size (233 KB). The maximum recommended task size is 100 KB.
23/03/07 22:05:49 WARN org.apache.spark.scheduler.TaskSetManager: Stage 297 contains a task of very large size (304 KB). The maximum recommended task size is 100 KB.
23/03/07 22:06:22 WARN org.apache.spark.scheduler.TaskSetManager: Stage 298 contains a task of very large size (233 KB). The maximum recommended task size is 100 KB.
23/03/07 22:06:22 WARN org.apache.spark.scheduler.TaskSetManager: Stage 299 contains a task of very large size (304 KB). The maximum recommended task size is 100 KB.


+--------------+----------+
|address_seq_no|   lat_lng|
+--------------+----------+
|    34.6970544|33.0560482|
|    34.6861583|33.0229121|
|    34.8099351|32.4230508|
|    35.1611988| 33.346177|
|    35.1023508|33.3720706|
|    35.1540521|33.3673642|
|     35.149884| 33.339715|
|    35.0608691|33.2597062|
|    34.6845316|33.0461744|
|    34.9155358|33.6142132|
|    34.6985076|33.0571559|
|    34.8464781|33.5727305|
|    35.1677609|33.3628195|
|    35.1616483|33.3719227|
|    35.1239299|33.2986405|
|    35.1728163|33.3503278|
|    35.0397493|33.9812021|
|    34.7117444|33.0186957|
|    34.6683242|33.0014804|
|    35.1686582|33.3635165|
+--------------+----------+
only showing top 20 rows



23/03/07 22:06:22 WARN org.apache.spark.scheduler.TaskSetManager: Stage 300 contains a task of very large size (233 KB). The maximum recommended task size is 100 KB.
23/03/07 22:06:22 WARN org.apache.spark.scheduler.TaskSetManager: Stage 301 contains a task of very large size (304 KB). The maximum recommended task size is 100 KB.
[Stage 301:======================================>                  (2 + 1) / 3]

+-----------------+-----------------+
|   address_seq_no|          lat_lng|
+-----------------+-----------------+
|       34.8049369|       33.4913536|
|       35.1582652|       33.3506226|
|        34.710771|        33.047564|
|34.77539489999999|       32.4217786|
|34.77539489999999|       32.4217786|
|       35.1750197|       33.3832332|
|       34.6884489|        33.032908|
|       35.1729524|       33.3808787|
|        35.157289|       33.3880389|
|       35.1855659|33.38227639999999|
|       34.6863359|       33.0517492|
|       35.0241012|       33.4290147|
|34.77539489999999|       32.4217786|
|       34.9182222|       33.6200625|
|       34.8049369|       33.4913536|
|       35.1695152|        33.359873|
|        35.060907|       33.9643725|
|       34.8929918|       33.6383893|
|       35.0203746|       33.7412559|
|             null|             null|
+-----------------+-----------------+
only showing top 20 rows



In [ ]:
a_5 = df[30001:45000]
addresses_df5 = spark.createDataFrame(a_5)
addresses_df5.show()
min_id5 = df['address_seq_no'][30001]
max_id5 = df['address_seq_no'][45000]
print(min_id5)
print(max_id5)

23/03/08 17:04:42 WARN org.apache.spark.scheduler.TaskSetManager: Stage 0 contains a task of very large size (234 KB). The maximum recommended task size is 100 KB.
[Stage 0:>                                                          (0 + 1) / 1]

+--------------+--------------------+
|address_seq_no|        Full_Address|
+--------------+--------------------+
|         69154|ΘΕΜ. ΔΕΡΒΗ 12 & Β...|
|         69155|ΝΙΚΗΣ 3  ΚΛΗΡΟΥ Λ...|
|         69156|25ΗΣ ΟΚΤΩΒΡΙΟΥ 25...|
|         69157|ΓΕΩΡΓΙΟΥ ΑΒΕΡΩΦ 5...|
|         69158|ΜΙΧΑΛΑΚΟΠΟΥΛΟΥ 14...|
|         69159|ΝΑΥΠΛΙΟΥ 5  ΑΡΑΔΙ...|
|         69161|ΑΡΧ. ΜΑΚΑΡΛΙΟΥ ΙΙ...|
|         69162|ΙΣΑΑΚΙΟΥ ΚΟΜΝΗΝΟΥ...|
|         69163|ΡΗΓΑ ΦΕΡΑΙΟΥ  ΛΕΜ...|
|         69164|ΛΕΜΕΣΟΥ - ΛΕΥΚΩΣΙ...|
|         69165|ΠΛΑΤΩΝΟΣ 4  ΑΓΙΑ ...|
|         69166|ΒΡΥΣΟΥΛΛΕΣ 161Α  ...|
|         69167|ΙΩΑΝΝΗ ΚΛΗΡΙΔΗ 2 ...|
|         69169| ΝΑΞΟΥ 4Ε  ΛΕΥΚΩΣΙΑ |
|         69170|ΑΡΧ. ΜΑΚΑΡΙΟΥ ΙΙΙ...|
|         69172|ΑΛΚΜΗΝΗΣ 9  ΚΑΤΩ ...|
|         69173|ΝΙΚΟΛΑΟΥ ΠΟΛΙΤΗ 5...|
|         69174|ΙΩΑΝΝΗ ΚΛΗΡΙΔΗ 2C...|
|         69176|ΟΜΟΝΟΙΑΣ 55  ΛΕΜΕ...|
|         69178|ΣΠΥΡΟΥ ΑΡΑΟΥΖΟΥ 6...|
+--------------+--------------------+
only showing top 20 rows

69154
123540


In [ ]:
offset51 = 69100
batch_size51 = 500

while (offset51<=max_id5):
    df_name51 = 'new_df_' + str(offset51)
    rdd_name51 = 'rdd_' + str(offset51)
    work_name51 = 'work_' + str(offset51)
    globals()[df_name51] = addresses_df5.filter((col("address_seq_no") >= offset51) & (col("address_seq_no") < (offset51+batch_size51)))
    globals()[rdd_name51] = globals()[df_name51].rdd.map(lambda x: get_coordinates2(x['address_seq_no'], x['Full_Address']))
    globals()[work_name51] = spark.createDataFrame(globals()[rdd_name51], schema=['address_seq_no', 'lat_lng'])
    globals()[work_name51].show()

    if offset51 > max_id5:
        break
    else:
        offset51 = offset51 + batch_size51

23/03/08 17:08:06 WARN org.apache.spark.scheduler.TaskSetManager: Stage 1 contains a task of very large size (234 KB). The maximum recommended task size is 100 KB.
23/03/08 17:08:07 WARN org.apache.spark.scheduler.TaskSetManager: Stage 2 contains a task of very large size (234 KB). The maximum recommended task size is 100 KB.
23/03/08 17:08:38 WARN org.apache.spark.scheduler.TaskSetManager: Stage 3 contains a task of very large size (234 KB). The maximum recommended task size is 100 KB.


+-----------------+----------+
|   address_seq_no|   lat_lng|
+-----------------+----------+
|              NaN|       NaN|
|       35.0213564|33.1777225|
|35.15105690000001|33.3121347|
|       34.9310206|33.6207682|
|        35.161803| 33.361042|
|       34.9298197|33.5995308|
|       34.7587825|32.9929063|
|        35.173114| 33.365803|
|       34.6909148|33.0509332|
|       34.6786322|33.0413055|
|       34.6867182|33.0437715|
|       35.0719751|33.8815901|
|       35.1637844|33.3697081|
|       35.1630664|33.3665425|
|       34.8049369|33.4913536|
|       34.7570703| 32.414901|
|       35.1267751|33.3511125|
|       35.1637844|33.3697081|
|       34.6689493|33.0129734|
|        34.673254|33.0446206|
+-----------------+----------+
only showing top 20 rows



23/03/08 17:08:38 WARN org.apache.spark.scheduler.TaskSetManager: Stage 4 contains a task of very large size (234 KB). The maximum recommended task size is 100 KB.
23/03/08 17:09:15 WARN org.apache.spark.scheduler.TaskSetManager: Stage 5 contains a task of very large size (234 KB). The maximum recommended task size is 100 KB.


+--------------+-----------------+
|address_seq_no|          lat_lng|
+--------------+-----------------+
|    35.1526719|       33.3647675|
|    35.0578648|       33.3886702|
|    34.6887314|       33.0500668|
|    35.1063639|       33.3782668|
|    34.6818842|       33.0460972|
|    34.7851587|       32.4316056|
|    35.1853986|         33.39069|
|    35.0369323|       33.9835815|
|     35.140179|       33.3359457|
|     34.673502|       33.0111371|
|    34.9288025|       33.6183202|
|    35.1731822|       33.3794196|
|    34.6857704|33.05635340000001|
|    35.0794909|       33.3262986|
|    34.9915973|       33.9971516|
|    34.9427024|       33.6287941|
|    35.1150978|       33.3801786|
|    35.1637531|       33.3330479|
|     35.158472|       33.3583131|
|    34.9915973|       33.9971516|
+--------------+-----------------+
only showing top 20 rows



23/03/08 17:09:16 WARN org.apache.spark.scheduler.TaskSetManager: Stage 6 contains a task of very large size (234 KB). The maximum recommended task size is 100 KB.
23/03/08 17:09:42 WARN org.apache.spark.scheduler.TaskSetManager: Stage 7 contains a task of very large size (234 KB). The maximum recommended task size is 100 KB.


...

23/03/08 17:22:10 WARN org.apache.spark.scheduler.TaskSetManager: Stage 116 contains a task of very large size (307 KB). The maximum recommended task size is 100 KB.
23/03/08 17:22:11 WARN org.apache.spark.scheduler.TaskSetManager: Stage 117 contains a task of very large size (234 KB). The maximum recommended task size is 100 KB.
23/03/08 17:22:11 WARN org.apache.spark.scheduler.TaskSetManager: Stage 118 contains a task of very large size (307 KB). The maximum recommended task size is 100 KB.
23/03/08 17:22:13 WARN org.apache.spark.scheduler.TaskSetManager: Stage 119 contains a task of very large size (234 KB). The maximum recommended task size is 100 KB.
23/03/08 17:22:13 WARN org.apache.spark.scheduler.TaskSetManager: Stage 120 contains a task of very large size (307 KB). The maximum recommended task size is 100 KB.


+--------------+----------+
|address_seq_no|   lat_lng|
+--------------+----------+
|    34.7809352|32.8266361|
|    35.1395696|33.3508226|
|     35.168248|33.3462739|
|    35.1698311|33.3570624|
|    34.7933336|32.4453083|
|    34.9164982|33.6242379|
|    34.9217529|33.6149208|
|    35.1635852|33.3591133|
|    34.6997318|33.0873408|
|    34.7593002|32.4554478|
|    35.1613008|33.3521217|
|    35.1670607|33.3473349|
|    35.0443276|33.9854941|
|    34.6786322|33.0413055|
|    34.9156365| 33.608558|
|    34.9490765|33.6275272|
|    34.6944614|33.0509292|
|    35.1590298|33.3669108|
|    34.9914416|33.9977504|
+--------------+----------+



23/03/08 17:22:14 WARN org.apache.spark.scheduler.TaskSetManager: Stage 121 contains a task of very large size (234 KB). The maximum recommended task size is 100 KB.
23/03/08 17:22:14 WARN org.apache.spark.scheduler.TaskSetManager: Stage 122 contains a task of very large size (307 KB). The maximum recommended task size is 100 KB.
23/03/08 17:22:15 WARN org.apache.spark.scheduler.TaskSetManager: Stage 123 contains a task of very large size (234 KB). The maximum recommended task size is 100 KB.
23/03/08 17:22:15 WARN org.apache.spark.scheduler.TaskSetManager: Stage 124 contains a task of very large size (307 KB). The maximum recommended task size is 100 KB.


+--------------+-----------------+
|address_seq_no|          lat_lng|
+--------------+-----------------+
|    35.1263103|       33.3168528|
|    34.6885991|       33.0554822|
|    34.7037117|       33.0423179|
|    34.6906647|       33.0686876|
|    35.1388557|       33.3667181|
|    34.9832317|         33.94044|
|    34.9182222|       33.6200625|
|    34.7765504|32.42953800000001|
|    34.9187359|       33.6014268|
|    34.9182222|       33.6200625|
|    34.6841379|       33.0419945|
|     34.752728|       32.4207792|
|    34.6788426|       33.0506093|
+--------------+-----------------+



ValueError: RDD is empty

In [ ]:
a_6 = df[45001:60000]
addresses_df6 = spark.createDataFrame(a_6)
addresses_df6.show()
min_id6 = df['address_seq_no'][45001]
max_id6 = df['address_seq_no'][60000]
print(min_id6)
print(max_id6)

23/03/08 20:41:30 WARN org.apache.spark.scheduler.TaskSetManager: Stage 0 contains a task of very large size (248 KB). The maximum recommended task size is 100 KB.
[Stage 0:>                                                          (0 + 1) / 1]

+--------------+--------------------+
|address_seq_no|        Full_Address|
+--------------+--------------------+
|        123550|ΣΟΦΟΥΛΗ 40  1096 ...|
|        123567|ΚΟΡΥΤΣΑΣ 12Α  308...|
|        123604|ΘΕΡΜΟΠΥΛΩΝ 7  ΓΕΡ...|
|        123608|ΟΔΥΣΣΕΙΑΣ 9  ΠΕΡΑ...|
|        123627|            ΛΕΜΕΣΟΣ |
|        123644|ΓΡ. ΑΥΞΕΝΤΙΟΥ  ΛΑ...|
|        123654|ΓΡΙΒΑ ΔΙΓΕΝΗ  330...|
|        123664|ΝΕΟΦΥΤΟΥ ΝΙΚΟΛΑΪΔ...|
|        123665|JOHN KENNEDY  310...|
|        123666|ΚΙΜΩΝΟΣ 52Α  ΑΓΙΟ...|
|        123671|ΣΟΥΛΙΟΥ 11  ΣΤΡΟΒ...|
|        123674|ΜΕΓΑΛΟΥ ΑΛΕΞΑΝΔΡΟ...|
|        123675|ΣΤΡΑΤΗΓΟΥ ΤΙΜΑΓΙΑ...|
|        123678|ΛΑΖΑΡΟΥ ΓΕΩΡΓΙΟΥ ...|
|        123679|ΧΡΙΣΤΑΚΗ ΚΡΑΝΟΥ 2...|
|        123686|ΣΤΑΣΙΝΟΥ 17  ΑΚΡΟ...|
|        123707|ΑΝΕΞΑΡΤΗΣΙΑΣ 56  ...|
|        123709|ΕΛΕΥΘΕΡΙΑΣ 82  ΧΛ...|
|        123710|ΑΓΙΑΣ 4  ΚΙΣΣΟΝΕΡ...|
|        123711|ΠΑΡΝΗΣ 18Α  ΑΓ. Φ...|
+--------------+--------------------+
only showing top 20 rows

123550
237118


In [ ]:
offset6 = 123500
batch_size6 = 1000

while (offset6<=max_id6):
    df_name6 = 'new_df_' + str(offset6)
    rdd_name6 = 'rdd_' + str(offset6)
    work_name6 = 'work_' + str(offset6)
    globals()[df_name6] = addresses_df6.filter((col("address_seq_no") >= offset6) & (col("address_seq_no") < (offset6+batch_size6)))
    globals()[rdd_name6] = globals()[df_name6].rdd.map(lambda x: get_coordinates2(x['address_seq_no'], x['Full_Address']))
    globals()[work_name6] = spark.createDataFrame(globals()[rdd_name6], schema=['address_seq_no', 'lat_lng'])
    globals()[work_name6].show()

    if offset6 > max_id6:
        break
    else:
        offset6 = offset6 + batch_size6

23/03/08 20:43:13 WARN org.apache.spark.scheduler.TaskSetManager: Stage 1 contains a task of very large size (248 KB). The maximum recommended task size is 100 KB.
23/03/08 20:43:14 WARN org.apache.spark.scheduler.TaskSetManager: Stage 2 contains a task of very large size (248 KB). The maximum recommended task size is 100 KB.
23/03/08 20:43:34 WARN org.apache.spark.scheduler.TaskSetManager: Stage 3 contains a task of very large size (248 KB). The maximum recommended task size is 100 KB.


+--------------+----------+
|address_seq_no|   lat_lng|
+--------------+----------+
|    35.1695107| 33.356824|
|    34.6880719|33.0359395|
|    34.7068302|33.0797917|
|    35.0164664|33.3991466|
|    34.6786322|33.0413055|
|    34.9084601|33.5964704|
|    34.6910989|33.0614018|
|    34.7743636| 32.428322|
|    34.6863594|33.0593261|
|    34.6903636|33.0523288|
|    35.1485645|33.3471325|
|     34.670786|32.9338168|
|          null|      null|
|    35.1449449|33.3550862|
|    34.6993192|33.0878985|
|    35.1686131|33.3640716|
|    34.9220437|32.4773453|
|    34.7936162|32.4137281|
|    34.8215192|32.3925906|
|    34.7056103|33.0268431|
+--------------+----------+
only showing top 20 rows



23/03/08 20:43:34 WARN org.apache.spark.scheduler.TaskSetManager: Stage 4 contains a task of very large size (248 KB). The maximum recommended task size is 100 KB.
23/03/08 20:43:52 WARN org.apache.spark.scheduler.TaskSetManager: Stage 5 contains a task of very large size (248 KB). The maximum recommended task size is 100 KB.


+-----------------+----------+
|   address_seq_no|   lat_lng|
+-----------------+----------+
|       35.1132937|33.3722216|
|       35.1739399|33.3605142|
|       34.8058577|32.4099308|
|        35.062854| 33.960628|
|       34.9220437|32.4773453|
|       34.6977382|32.9639052|
|       35.1339703|33.0351108|
|34.77539489999999|32.4217786|
|             null|      null|
|       34.6786322|33.0413055|
|       34.6892466|33.0688915|
|       34.9198992|33.6316196|
|       34.7084494| 33.125119|
|       35.1665073|33.3711122|
|       34.7988017|32.4084312|
|       35.0019218|33.3710143|
|             null|      null|
|       34.6801684|33.0531576|
|       35.1647383|33.3162112|
|       35.1651652| 33.372919|
+-----------------+----------+
only showing top 20 rows



23/03/08 20:43:53 WARN org.apache.spark.scheduler.TaskSetManager: Stage 6 contains a task of very large size (248 KB). The maximum recommended task size is 100 KB.
23/03/08 20:44:08 WARN org.apache.spark.scheduler.TaskSetManager: Stage 7 contains a task of very large size (248 KB). The maximum recommended task size is 100 KB.


...

+-----------------+-----------------+
|   address_seq_no|          lat_lng|
+-----------------+-----------------+
|       34.7178216|       33.0786471|
|       35.0339441|       32.4253751|
|       34.7477754|         32.42564|
|         34.67591|       33.0048501|
|34.77231810000001|       32.4097377|
|       34.9947064|       33.9778287|
|       35.0617995|       33.5399098|
|       35.0375882|       33.9814728|
|             null|             null|
|       35.0510457|       33.9926992|
|             null|             null|
|       34.8232973|       32.4019769|
|       34.6847414|       33.0343849|
|       41.1079438|       25.7168991|
|         34.99264|       34.0004834|
|       35.0120718|       34.0562252|
|34.76140669999999|       32.4127915|
|       35.1642754|        33.368012|
|       35.1611567|33.36614290000001|
|35.16544349999999|       33.3452223|
+-----------------+-----------------+
only showing top 20 rows



23/03/08 21:07:46 WARN org.apache.spark.scheduler.TaskSetManager: Stage 399 contains a task of very large size (248 KB). The maximum recommended task size is 100 KB.
23/03/08 21:07:46 WARN org.apache.spark.scheduler.TaskSetManager: Stage 400 contains a task of very large size (330 KB). The maximum recommended task size is 100 KB.
23/03/08 21:07:57 WARN org.apache.spark.scheduler.TaskSetManager: Stage 401 contains a task of very large size (248 KB). The maximum recommended task size is 100 KB.
23/03/08 21:07:57 WARN org.apache.spark.scheduler.TaskSetManager: Stage 402 contains a task of very large size (330 KB). The maximum recommended task size is 100 KB.


+-----------------+----------+
|   address_seq_no|   lat_lng|
+-----------------+----------+
|       35.1727775|33.3845084|
|        34.913474|33.6294647|
|       34.6899817|33.0464402|
|       34.9893934|33.9978441|
|       35.1475837|33.3553187|
|       34.9828802|33.6558907|
|       34.7587825|32.9929063|
|34.77539489999999|32.4217786|
|       35.1374078|33.3607413|
|       34.8388506|32.4348263|
|       34.9347489|33.6261286|
|        35.135548|33.3360237|
|        34.979849|33.9300723|
|       35.0096685|33.8929049|
|        35.140054|33.3632789|
|       34.9152406|33.6173991|
|       34.8989432|33.6385948|
|       35.1729518|33.3808814|
|       34.9155446|33.6287883|
|       34.6839069|33.0530057|
+-----------------+----------+
only showing top 20 rows



23/03/08 21:07:58 WARN org.apache.spark.scheduler.TaskSetManager: Stage 403 contains a task of very large size (248 KB). The maximum recommended task size is 100 KB.
23/03/08 21:07:58 WARN org.apache.spark.scheduler.TaskSetManager: Stage 404 contains a task of very large size (330 KB). The maximum recommended task size is 100 KB.
[Stage 404:======================================>                  (2 + 1) / 3]

+-----------------+----------+
|   address_seq_no|   lat_lng|
+-----------------+----------+
|35.15544879999999|33.3297917|
|       35.0359049|33.7817622|
|       34.7023294|33.0481908|
|       35.1452815|33.3428112|
|        34.764846|32.4188311|
|        34.986253|34.0003781|
|       35.1499663| 33.338858|
|       35.1665136|33.3643962|
|       34.7610466|32.4480824|
|       35.1735357|33.3648099|
|       34.8049369|33.4913536|
|       35.1448748|33.3794717|
|       35.1573322|33.3535815|
|       34.9521888|33.5908529|
|        35.132529|33.3322136|
|       35.1703529| 33.364985|
|       35.2114584|  33.69029|
|       35.1515982|33.3717682|
|       34.7587825|32.9929063|
|       34.8938825|32.8365745|
+-----------------+----------+
only showing top 20 rows



In [ ]:
# Authenticate and set the API Key
apikey = 'AIzaxxxxxxxxxxxxxxxxxxxxxxxxx'

with open('output2_nan.csv', 'a', encoding='utf-8') as g:
    writer = csv.writer(g)
    writer.writerow(['seq_no', 'coordinates', 'lat', 'lng'])
    
with open('output2_coords.csv', 'a', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['seq_no', 'coordinates', 'lat', 'lng'])

def get_coordinates2(seq_no, address):

    url = f'https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={apikey}' 
    response = requests.get(url).json() 
    if response['status'] == 'ZERO_RESULTS':
        lat = 'NaN'
        lng = 'NaN'
        with open('output2_nan.csv', 'a', encoding='utf-8') as g:
            writer = csv.writer(g)
            writer.writerow([seq_no, (lat, lng), lat, lng])
        return lat, lng
    else:
        lat = response['results'][0]['geometry']['location']['lat']
        lng = response['results'][0]['geometry']['location']['lng']
        with open('output2_coords.csv', 'a', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow([seq_no, (lat, lng), lat, lng])
        return lat, lng



In [ ]:
a_7 = df[60001:75000]
addresses_df7 = spark.createDataFrame(a_7)
addresses_df7.show()
min_id7 = df['address_seq_no'][60001]
max_id7 = df['address_seq_no'][75000]
print(min_id7)
print(max_id7)

+--------------+--------------------+
|address_seq_no|        Full_Address|
+--------------+--------------------+
|        237126|ΣΤΑΣΙΝΟΥ 28  ΛΕΥΚ...|
|        237144|ΚΩΝΣΤΑΝΤΙΝΟΥ & ΕΛ...|
|        237161|ΚΑΛΟΓΡΑΙΩΝ 16  ΛΑ...|
|        237168|ΔΗΜΗΤΣΑΝΗΣ 15  10...|
|        237172|ΤΕΥΚΡΟΥ 5  ΛΕΥΚΩΣΙΑ |
|        237183|ΚΥΚΛΑΔΩΝ 112Α  ΑΝ...|
|        237197|ΔΙΟΡΙΟΥ 6  ΓΕΡΙ 2...|
|        237203|ΣΩΤΗΡΙΟΥ ΤΟΦΙΝΗ 2...|
|        237217|Θ. ΚΟΛΟΚΟΤΡΩΝΗ 25...|
|        237218|ΚΥΡΙΑΚΟΥ ΜΑΤΣΗ 23...|
|        237221|ΕΙΡΗΝΗΣ 111  3041...|
|        237222|ΕΥΡΩΠΗΣ 20  ΣΤΡΟΒ...|
|        237223|ΑΜΑΘΟΥΝΤΑΣ 17  ?Γ...|
|        237224|ΠΙΝΔΑΡΟΥ 16  ΑΘΗΑ...|
|        237225|ΔΗΜΟΣΘΕΝΟΥΣ 1Α  3...|
|        237228|ΑΡΝΟΛΤ 9  3075 ΛΕ...|
|        237231|ΑΡΧ. ΜΑΚΑΡΙΟΥ ΙΙΙ...|
|        237234|ΑΓΙΑΣ ΦΥΛΑΞΕΩΣ & ...|
|        237235|ΗΠΕΙΡΟΥ 17  6031 ...|
|        237238|ΛΑΡΝΑΚΑΣ 23  ΨΕΥΔ...|
+--------------+--------------------+
only showing top 20 rows

237126
279984


23/03/08 21:14:48 WARN org.apache.spark.scheduler.TaskSetManager: Stage 408 contains a task of very large size (245 KB). The maximum recommended task size is 100 KB.


In [ ]:
offset7 = 237100
batch_size7 = 1000

while (offset7<=max_id7):
    df_name7 = 'new_df_' + str(offset7)
    rdd_name7 = 'rdd_' + str(offset7)
    work_name7 = 'work_' + str(offset7)
    globals()[df_name7] = addresses_df7.filter((col("address_seq_no") >= offset7) & (col("address_seq_no") < (offset7+batch_size7)))
    globals()[rdd_name7] = globals()[df_name7].rdd.map(lambda x: get_coordinates2(x['address_seq_no'], x['Full_Address']))
    globals()[work_name7] = spark.createDataFrame(globals()[rdd_name7], schema=['address_seq_no', 'lat_lng'])
    globals()[work_name7].show()

    if offset7 > max_id7:
        break
    else:
        offset7 = offset7 + batch_size7

23/03/08 21:15:39 WARN org.apache.spark.scheduler.TaskSetManager: Stage 410 contains a task of very large size (245 KB). The maximum recommended task size is 100 KB.
23/03/08 21:15:39 WARN org.apache.spark.scheduler.TaskSetManager: Stage 411 contains a task of very large size (245 KB). The maximum recommended task size is 100 KB.
23/03/08 21:15:54 WARN org.apache.spark.scheduler.TaskSetManager: Stage 412 contains a task of very large size (245 KB). The maximum recommended task size is 100 KB.


+-----------------+----------+
|   address_seq_no|   lat_lng|
+-----------------+----------+
|35.16886290000001|33.3677183|
|             null|      null|
|       34.9184068|33.6350026|
|       35.1638786|33.3675947|
|       35.1645793| 33.358121|
|       35.1228026|33.2903736|
|       35.1011528|33.4157833|
|       34.6998641|33.0605313|
|       34.6959569|32.9445049|
|       35.1602093|33.3561009|
|       34.6746399|33.0410171|
|       35.1575787|33.3437863|
|       34.7258619|33.1404129|
|       34.9182222|33.6200625|
|       34.6765061|33.0374392|
|       34.6786322|33.0413055|
|       35.0406689|33.9852053|
|       34.6922876|33.0352302|
|       34.9085807|33.6275589|
|       34.9512771|33.4631238|
+-----------------+----------+
only showing top 20 rows



23/03/08 21:15:55 WARN org.apache.spark.scheduler.TaskSetManager: Stage 413 contains a task of very large size (245 KB). The maximum recommended task size is 100 KB.
23/03/08 21:16:03 WARN org.apache.spark.scheduler.TaskSetManager: Stage 414 contains a task of very large size (245 KB). The maximum recommended task size is 100 KB.


+-----------------+-----------------+
|   address_seq_no|          lat_lng|
+-----------------+-----------------+
|       35.1727869|       33.3290916|
|       34.7988017|       32.4084312|
|       35.1666181|       33.3307245|
|       35.1535039|       33.3750821|
| 34.9123268817192|33.59883469681587|
|       35.1507278|       33.3732429|
|       35.1602214|       33.3604734|
|        37.976277|       23.7290321|
|       34.6794128|       33.0310024|
|       34.7877498|        32.429265|
|       34.6820046|       33.0369246|
|       34.6882836|       32.9612103|
|       34.7987765|       33.3836411|
|       35.1397142|       33.3183216|
|       35.1655348|       33.3640062|
|35.18712600000001|       33.3884794|
|       35.1658285|       33.3609257|
|       34.9761878|       33.8487527|
|       35.1681386|       33.3177874|
|       39.1896509|       20.9515767|
+-----------------+-----------------+
only showing top 20 rows



23/03/08 21:16:04 WARN org.apache.spark.scheduler.TaskSetManager: Stage 415 contains a task of very large size (245 KB). The maximum recommended task size is 100 KB.
23/03/08 21:16:22 WARN org.apache.spark.scheduler.TaskSetManager: Stage 416 contains a task of very large size (245 KB). The maximum recommended task size is 100 KB.


...

+--------------+-----------------+
|address_seq_no|          lat_lng|
+--------------+-----------------+
|    35.1645602|       33.3721419|
|    34.9152417|       33.6312687|
|    35.3169997|       33.9457337|
|    35.1855659|33.38227639999999|
|    35.1149116|        33.919245|
|    35.1615504|       33.3627817|
|    35.1775642|       33.3703957|
|    35.1855659|33.38227639999999|
|     35.161304|       33.3861164|
|    35.1855659|33.38227639999999|
|    35.0386896|       33.9270345|
|     34.681667|        33.048377|
|    35.1740496|       33.3616675|
|      35.17388|       33.3622956|
|    35.1715372|       33.3263174|
|    35.1855659|33.38227639999999|
|    35.0339441|       32.4253751|
|    35.1149116|        33.919245|
|    35.1094602|       33.3020529|
|    35.1855659|33.38227639999999|
+--------------+-----------------+
only showing top 20 rows



23/03/08 21:38:33 WARN org.apache.spark.scheduler.TaskSetManager: Stage 552 contains a task of very large size (245 KB). The maximum recommended task size is 100 KB.
23/03/08 21:38:33 WARN org.apache.spark.scheduler.TaskSetManager: Stage 553 contains a task of very large size (311 KB). The maximum recommended task size is 100 KB.
23/03/08 21:38:46 WARN org.apache.spark.scheduler.TaskSetManager: Stage 554 contains a task of very large size (245 KB). The maximum recommended task size is 100 KB.
23/03/08 21:38:46 WARN org.apache.spark.scheduler.TaskSetManager: Stage 555 contains a task of very large size (311 KB). The maximum recommended task size is 100 KB.


+-----------------+-----------------+
|   address_seq_no|          lat_lng|
+-----------------+-----------------+
|       34.7081603|       33.1249924|
|       35.1750561|       33.3649109|
|       34.6946276|       33.0393926|
|       35.1639168|       33.3590914|
|       35.1677897|       33.3682717|
|       35.1149116|        33.919245|
|       35.1639168|       33.3590914|
|       35.0375332|        33.913204|
|       35.1639168|       33.3590914|
|       35.1268061|       33.1921206|
|       34.9247273|        33.626126|
|       35.1436629|       33.3437909|
|35.15870400000001|        33.356451|
|        34.700547|           33.011|
|       35.1855659|33.38227639999999|
|       35.1734134|       33.3637723|
|       35.1855659|33.38227639999999|
|       34.6749065|       33.0379886|
|       35.1149116|        33.919245|
|       34.6741744|       33.0439418|
+-----------------+-----------------+
only showing top 20 rows



23/03/08 21:38:46 WARN org.apache.spark.scheduler.TaskSetManager: Stage 556 contains a task of very large size (245 KB). The maximum recommended task size is 100 KB.
23/03/08 21:38:46 WARN org.apache.spark.scheduler.TaskSetManager: Stage 557 contains a task of very large size (311 KB). The maximum recommended task size is 100 KB.
[Stage 557:======================================>                  (2 + 1) / 3]

+--------------+----------+
|address_seq_no|   lat_lng|
+--------------+----------+
|    35.1855919| 33.397311|
|    34.9220941|33.5915213|
|    34.7569481|32.4125481|
|    34.6780345|33.0442249|
|          null|      null|
|          null|      null|
|    34.7931742|32.4156112|
|    35.1478279|33.3988237|
|     35.240117|24.8092691|
|     35.151853| 33.374953|
|    35.1494669|33.3175654|
|    38.4387082|22.8753993|
|    34.7652152|32.4112877|
|          null|      null|
|    35.1108253|33.3721974|
|    35.1729524|33.3808787|
|    35.1615276|33.3561984|
|    35.1866902|33.3760571|
|    35.1559835|33.3583682|
|    35.3399081|33.3174035|
+--------------+----------+
only showing top 20 rows



In [ ]:
a_8 = df[75001:90000]
addresses_df8 = spark.createDataFrame(a_8)
addresses_df8.show()
min_id8 = df['address_seq_no'][75001]
max_id8 = df['address_seq_no'][90000]
print(min_id8)
print(max_id8)

In [ ]:
offset8 = 237100
batch_size8 = 1000

while (offset8<=max_id8):
    df_name8 = 'new_df_' + str(offset8)
    rdd_name8 = 'rdd_' + str(offset8)
    work_name8 = 'work_' + str(offset8)
    globals()[df_name8] = addresses_df8.filter((col("address_seq_no") >= offset8) & (col("address_seq_no") < (offset8+batch_size8)))
    globals()[rdd_name8] = globals()[df_name8].rdd.map(lambda x: get_coordinates2(x['address_seq_no'], x['Full_Address']))
    globals()[work_name8] = spark.createDataFrame(globals()[rdd_name8], schema=['address_seq_no', 'lat_lng'])
    globals()[work_name8].show()

    if offset8 > max_id8:
        break
    else:
        offset8 = offset8 + batch_size8